# Party time #

In this tutorial we will look at the classes that put the "party" in `sipparty`.

"Party" here means a person who wants to make SIP calls (or participate in other types of "Session" via the "Session Initiation Protocol"). `Party` is the highest-level object in `sipparty`.

## Types of party ##

So, let's see what types of party we can have with `sipparty`.

In [ ]:
from sipparty.parties import AllPartyTypes
AllPartyTypes

Let's start with the `NoMediaSimpleCallsParty`. This is a demonstration type that doesn't include any media session, so it is just used for demonstrating the SIP signalling flow. 

## Parties listening ##

The first thing to do with the party is start listening, so that we can receive calls.

In [ ]:
from sipparty.parties import NoMediaSimpleCallsParty
NoMediaSimpleCallsParty
p1 = NoMediaSimpleCallsParty()
p1.listen()

But we can't just listen, because part of listening is to register the party's SIP identity with the transport component so that the party can be addressed. To do this, we need to configure a SIP URI on the party. Try doing so below:

In [ ]:
p1.uri = 'sip:me2@myhost.com'
p1.uri


The key thing to note here is that while we assigned a string to `p1.uri`, `sipparty` provides a mechanism for automatically parsing properties into deep object graphs via the `sipparty.parse.ParsedProperty` descriptor. If you were to try 

    p1.uri = 'notavalidURI-as-no-protocol-info'
    
You would get an error.

Continuing, now we will be able to listen (the listen can take a few seconds depending on the host). By default we listen on port 5060, since this is the standard SIP port.

In [ ]:
p1.listen()
p1.contactURI.port

We could in fact have overridden the default, and even provided a range of ports we'd have been happy with using a filter.

    p1.listen(port=None, port_filter=lambda x: x % 2 == 0)
    p1.contactURI.port
    
However, if you try this straight away you'll get an error: once you're listening you have to unlisten before you can relisten, using:

    p1.stop_listen()
    
[!! Not yet implemented !!]

## Start a call. ##

Now you've got a party, you could start a call to a remote person. However, for this demo we don't have a remote person to call, so we'll just have to create a new party and call that instead.

In [ ]:
p2 = NoMediaSimpleCallsParty(aor='you@yourhost.com')
dlg = p2.invite(p1)

> Note how we can specify the Address Of Record for the new party in the initialization call. The AOR is part of the URI, and `Party` classes will automatically delegate their AOR attribute through to their URI. So:
>
>     p1.uri.aor = 'me@myhost.com'
>
> is equivalent to:
>
>     p1.aor = 'me@myhost.com'
>
> The classes `sipparty.deepclass.DeepClass` and `sipparty.vb.ValueBinder` are used to achieve this.

## Dialogs

The `p1.invite()` method returns a `Dialog` instance which maintains the state of a particular call. Now if you want to manipulate that call, you can act on that. The `Dialog` instances are all independent, and you can have multiple dialogs per call. Let's create another one:

In [ ]:
dlg2 = p2.invite(p1)

Dialogs all act asynchronously. So the state of them may change at any time. See what state your dialogs are in now.

In [ ]:
print(dlg.state)
print(dlg2.state)

They should both be `InDialog` (you may have to wait a second). 

## Terminate

Terminate one of them now:

In [ ]:
dlg2.terminate()
dlg.state

It might take a couple of seconds, but the state should move to `Terminated`. 

In [ ]:
dlg2.state

For curiosity's sake, see what happens if you terminate again.

In [ ]:
dlg2.terminate()

## FSMs

You should receive the `sipparty.fsm.UnexpectedInput` exception. This illustrates that the `Dialog` classes inherit from a class called `sipparty.fsm.FSM`, which implements Finite State Machines. This provides a convenient way to control state. The definition of the state transitions that the `Dialog` can take are defined in the concrete subclasses of the `Dialog`. You can see this by calling the FSM string method:

In [ ]:
from sipparty.fsm import FSM
print(FSM.__str__(dlg))

So let's finish up by closing the other dialog.

In [ ]:
dlg.terminate()

Again, you may have to wait a couple of seconds for the call to terminate.

In [ ]:
dlg.state

## Finished

That concludes this tutorial. In the next one we'll look at how you can hook into the dialogs and parties to take more control over the calls.